In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import os


In [2]:
# load model
model_path = "/Users/zhenyiye/PycharmProjects/mbfacenet_tf2/utils/mbfacenet_wo_prelu.h5"
model = keras.models.load_model(model_path)

In [3]:
# freeze model
model.trainable = False

In [4]:
# define a finetune header
num_class = 2

inputs = model.input
embeds = model(inputs)
x = keras.layers.Dense(128, activation='relu')(embeds)
x = keras.layers.Dense(256, activation='relu')(x)
output = keras.layers.Dense(num_class, activation='softmax', name='cls_output')(x)

finetune_model = keras.Model(inputs=inputs, outputs=[embeds, output])

In [5]:
# examine model
finetune_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 112, 96, 3)]      0         
_________________________________________________________________
model (Model)                (None, 128)               1011584   
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
cls_output (Dense)           (None, 2)                 514       
Total params: 1,061,634
Trainable params: 50,050
Non-trainable params: 1,011,584
_________________________________________________________________


In [6]:
finetune_model.compile(optimizer='adam', loss={'cls_output': 'categorical_crossentropy'}, metrics=['accuracy'])

In [7]:
# load dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

data_path = "/home/yzy/Downloads/Face-Mask-Detection-master/dataset"
def load_mask_dataset():
    X, y = [], []
    for root, dirs, filenames in os.walk(data_path):
        for file in filenames:
            if os.path.splitext(file)[-1] == '.png' or os.path.splitext(file)[-1] == '.jpg':
                X.append(os.path.join(root, file))
                y.append(os.path.split(root)[-1])
    le = LabelEncoder()
    y = le.fit_transform(y)
    print(le.classes_)
    trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.1)
    return trainX, testX, trainy, testy

def preprocess(x, y):
    # x: directory，y：label
    x = tf.io.read_file(x)
    x = tf.image.decode_jpeg(x, channels=3) # RGBA
    x = tf.image.resize(x, [112, 96])

    x = tf.image.random_flip_left_right(x)

    # x: [0,255]=> -1~1
    x = (tf.cast(x, dtype=tf.float32) - 127.5) / 128.0
    y = tf.convert_to_tensor(y)
    y = tf.one_hot(y, depth=2)

    return x, y

# get data slices
train_image, val_image, train_label, val_label = load_mask_dataset()

['with_mask' 'without_mask']


In [8]:
# construct input pipeline
batchsz = 128
db_train = tf.data.Dataset.from_tensor_slices((train_image, train_label))     # construct train dataset
db_train = db_train.shuffle(1000).map(preprocess).batch(batchsz)
db_val = tf.data.Dataset.from_tensor_slices((val_image, val_label))
db_val = db_val.shuffle(1000).map(preprocess).batch(batchsz)

In [9]:
# training
finetune_model.fit(db_train, validation_data=db_val, validation_freq=1, epochs=10)

Epoch 1/10
27/27 [==============================] - 71s 3s/step - loss: 1.1615 - cls_output_loss: 0.5437 - model_accuracy: 0.0110 - cls_output_accuracy: 0.7772 - val_loss: 0.9978 - val_cls_output_loss: 0.3799 - val_model_accuracy: 0.0053 - val_cls_output_accuracy: 0.8720

KeyboardInterrupt: 

In [11]:
# evaluation
import numpy as np
correct = 0
all = 0
for x, y in db_val:
    prediction = finetune_model.predict(x)
    prediction = np.argmax(prediction[1], axis=1)
    correct += np.sum(prediction == np.argmax(y, axis=1))
    all += x.shape[0]
print(correct/all)

In [20]:
keras.models.save_model(finetune_model, "model_with_mask_clf.h5")

In [26]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 112, 96, 3)] 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 114, 98, 3)   0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 56, 48, 64)   1728        zero_padding2d[0][0]             
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 56, 48, 64)   256         conv2d[0][0]                     
______________________________________________________________________________________________